### Sentiment Analysis - LSTM (Long Short Term Memory)

#### Import Needed Package

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#### Load Dataset

In [2]:
data = pd.read_csv('IMDB_Dataset.csv') # read locally

data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data.shape

(50000, 2)

In [4]:
type(data)

pandas.core.frame.DataFrame

In [5]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [6]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


#### One Hot Encoding

In [7]:
# Label Encoder
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)

In [8]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### Data Preprocessing

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [11]:
train_data.shape

(40000, 2)

In [12]:
test_data.shape

(10000, 2)

In [13]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

In [14]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [15]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [16]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [17]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

### Model Building

In [18]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [21]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 349s 689ms/step - accuracy: 0.7347 - loss: 0.5187 - val_accuracy: 0.8540 - val_loss: 0.3363
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 334s 669ms/step - accuracy: 0.8519 - loss: 0.3593 - val_accuracy: 0.8569 - val_loss: 0.3365
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 327s 655ms/step - accuracy: 0.8698 - loss: 0.3214 - val_accuracy: 0.8594 - val_loss: 0.3609
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 330s 661ms/step - accuracy: 0.8979 - loss: 0.2617 - val_accuracy: 0.8303 - val_loss: 0.3727
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 377s 650ms/step - accuracy: 0.8952 - loss: 0.2561 - val_accuracy: 0.8556 - val_loss: 0.3544


In [22]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 85ms/step - accuracy: 0.8606 - loss: 0.3388


In [ ]:
print(loss)

In [24]:
print(accuracy)

0.8593999743461609


### Building predictive system

In [25]:
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequences, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment  = "Positive" if prediction[0][0] >= 0.5 else "Negative"
    return sentiment


In [26]:
predictive_system("This movie is great!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step


'Positive'

In [27]:
predictive_system("UIB is a great campus and I love it")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


'Positive'

In [28]:
# Save Model
model.save("tugas.h5")

import joblib
joblib.dump(tokenizer, 'tokenizer.pkl') # dump tokenizer

['tokenizer.pkl']